<a href="https://colab.research.google.com/github/BryanSouza91/COVID-19/blob/master/COVID19_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf
tf.__version__

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
tf.random.set_seed(29)

NameError: name 'tf' is not defined

In [4]:
confirmed_url = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
recovered_url = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
deaths_url = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

In [5]:
conf_df = pd.read_csv(confirmed_url)
recv_df = pd.read_csv(recovered_url)
death_df = pd.read_csv(deaths_url)

In [6]:
print(conf_df.loc[:,'1/22/20':].sum().tail())
print(recv_df.loc[:,'1/22/20':].sum().tail())
print(death_df.loc[:,'1/22/20':].sum().tail())

6/25/20     9610551
6/26/20     9802294
6/27/20     9980257
6/28/20    10145791
6/29/20    10302052
dtype: int64
6/25/20    4838921
6/26/20    4945557
6/27/20    5051864
6/28/20    5140899
6/29/20    5235813
dtype: int64
6/25/20    489312
6/26/20    494181
6/27/20    498710
6/28/20    501893
6/29/20    505505
dtype: int64


In [7]:
print("New cases as of {}: {}".format(conf_df.columns[-1],(conf_df.loc[::].sum()[-1] - conf_df.loc[::].sum()[-2])))
print("New case rate: {0:.3%}".format((conf_df.loc[::].sum()[-1] - conf_df.loc[::].sum()[-2])/ conf_df.loc[::].sum()[-2]))
print("Death rate: {0:.3%}".format(death_df.loc[::].sum()[-1]/conf_df.loc[::].sum()[-1]))

New cases as of 6/29/20: 156261
New case rate: 1.540%
Death rate: 4.907%


In [8]:
def standardize(x, mean, std):
  return (x - mean) / std

def inv_standardize(x, mean, std):
  return (x * std) + mean

In [9]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [10]:
# define input sequence

raw_seq = np.log10(conf_df.loc[:,'1/22/20':].sum().values)

# get mean and std
TRAIN_SPLIT = int(len(raw_seq)*.8)
print(TRAIN_SPLIT)
MEAN = raw_seq[:TRAIN_SPLIT].mean()
STD = raw_seq[:TRAIN_SPLIT].std()
stand_seq = standardize(raw_seq, MEAN, STD)

# choose a number of time steps

n_steps_in, n_steps_out = 9, 3

# split into samples

X, y = split_sequence(stand_seq, n_steps_in, n_steps_out)

for i in range(len(X)):
  print(X[i], y[i])

128
[-2.82508695 -2.75342463 -2.59457586 -2.4106473  -2.24037341 -2.09913251
 -1.81759344 -1.7738379  -1.64756529] [-1.56592802 -1.4817482  -1.33656455]
[-2.75342463 -2.59457586 -2.4106473  -2.24037341 -2.09913251 -1.81759344
 -1.7738379  -1.64756529 -1.56592802] [-1.4817482  -1.33656455 -1.26271032]
[-2.59457586 -2.4106473  -2.24037341 -2.09913251 -1.81759344 -1.7738379
 -1.64756529 -1.56592802 -1.4817482 ] [-1.33656455 -1.26271032 -1.18247331]
[-2.4106473  -2.24037341 -2.09913251 -1.81759344 -1.7738379  -1.64756529
 -1.56592802 -1.4817482  -1.33656455] [-1.26271032 -1.18247331 -1.11893157]
[-2.24037341 -2.09913251 -1.81759344 -1.7738379  -1.64756529 -1.56592802
 -1.4817482  -1.33656455 -1.26271032] [-1.18247331 -1.11893157 -1.07167597]
[-2.09913251 -1.81759344 -1.7738379  -1.64756529 -1.56592802 -1.4817482
 -1.33656455 -1.26271032 -1.18247331] [-1.11893157 -1.07167597 -1.02344316]
[-1.81759344 -1.7738379  -1.64756529 -1.56592802 -1.4817482  -1.33656455
 -1.26271032 -1.18247331 -1.118

In [11]:
# reshape from [samples, timesteps] into [samples, timesteps, features]

n_features = 1

X = X.reshape((X.shape[0], X.shape[1], n_features))
X.shape

(149, 9, 1)

In [12]:
# define model

def get_compiled_model(n_steps_in, n_steps_out, n_features):
  model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(96, 2, activation='elu', input_shape=(n_steps_in, n_features)),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(32, 2, activation='elu'),
    tf.keras.layers.MaxPooling1D(1),
    tf.keras.layers.Conv1D(32, 1, activation='elu'),
    tf.keras.layers.MaxPooling1D(1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='elu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Dense(n_steps_out)
  ])

  model.compile(optimizer='adam', loss='mae')

  return model

In [13]:
model = get_compiled_model(n_steps_in,n_steps_out,n_features)

model.summary()

NameError: name 'tf' is not defined

In [14]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=.9, patience=6, min_lr=1e-7, verbose=1)

NameError: name 'tf' is not defined

In [15]:
# fit model
hist = model.fit(X, y, validation_split=.2, epochs=200, verbose=1, shuffle=True, callbacks=[early_stop])

NameError: name 'model' is not defined

In [16]:
# demonstrate prediction
x_input = np.log10(conf_df.loc[:,'1/22/20':].sum()[-(n_steps_in+1):-1].to_numpy())
x_inMEAN = x_input.mean()
x_inSTD = x_input.std()
x_input = standardize(x_input, x_inMEAN, x_inSTD)
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
yhat = inv_standardize(yhat, x_inMEAN, x_inSTD)
print("Prediction: {0:.0f} True: {1:.0f}".format((10 ** yhat[0][-1]), (10 ** raw_seq[-1])))
PE = (((10 ** yhat[0][-1] - 10 ** raw_seq[-1])/10 ** raw_seq[-1]))
print("Percent Error: {0:.3%}".format(PE))

NameError: name 'model' is not defined

In [17]:
plt.figure(figsize=(25,8))
sns.lineplot([n for n in range(1,3)],[10**n for n in yhat[0][:2]]);
sns.lineplot([n for n in range(1,3)],conf_df.loc[:,'1/22/20':].sum()[-3:].to_numpy()[1:]);

NameError: name 'yhat' is not defined

In [18]:
# prediction for tomorrow
x_input = np.log10(conf_df.loc[:,'1/22/20':].sum()[-n_steps_in:].to_numpy())
x_inMEAN = x_input.mean()
x_inSTD = x_input.std()
x_input = standardize(x_input, x_inMEAN, x_inSTD)
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
yhat = inv_standardize(yhat, x_inMEAN, x_inSTD)
print("%d" % (10 ** yhat[0][-1]))

NameError: name 'model' is not defined

In [19]:
for pred in yhat[0]:
  print("%d" % (10 ** pred))

NameError: name 'yhat' is not defined

In [20]:
hist = pd.DataFrame(hist.history)

NameError: name 'hist' is not defined

In [21]:
hist.plot(figsize=(25,6));

NameError: name 'hist' is not defined

In [22]:
[n for n in range(1,4)],[10**n for n in yhat[0]],conf_df.loc[:,'1/22/20':].sum()[-3:].to_numpy()

NameError: name 'yhat' is not defined